# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [7]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: False


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).

For your convenience, we provide the dataloader for you.


In [8]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=0, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=0, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=0, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [9]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75,100)
        self.recurrent_layer = nn.LSTM(100,100)
        
        self.classify_layer = nn.Linear(100,10)
        ###################################################
        
        
       
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        #print(hn)
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [10]:
################ 2nd To Do  (5 points)##################
#optimizer = torch.optim.RMSprop(advanced_model.parameters(), lr=0.0003)
optimizer=torch.optim.SGD(model.parameters(), lr = 1e-3)


criterion = torch.nn.CrossEntropyLoss()

In [11]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.3121,  Accuracy: 9.95
val Epoch: 0  , Loss: 2.3099,  Accuracy: 9.75


In [12]:
# Use your best model to generate results on test set.
# advanced model
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)

class SequenceClassify1(nn.Module):
    def __init__(self):
        super(SequenceClassify1, self).__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv2d(1,400, kernel_size=2, stride=1),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.02),
            nn.BatchNorm2d(400),
            nn.Dropout(0.2),
            nn.Conv2d(400, 300, kernel_size=2, stride=1),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.02),
            nn.BatchNorm2d(300),
            nn.Dropout(0.2),
            nn.Conv2d(300,200, kernel_size=2, stride=1),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.02),
            nn.BatchNorm2d(200),
            Flatten(),
            nn.Linear(1600, 200))
        self.recurrent_layer = nn.LSTM(200,400,num_layers = 2,dropout=0.2,bidirectional=True,batch_first=True)
        self.classify_layer=nn.Sequential(
        nn.Linear(800,400),
        nn.LeakyReLU(0.02),
        nn.BatchNorm1d(400),
        nn.Linear(400,128),
        nn.LeakyReLU(0.02),
        nn.BatchNorm1d(128),
        nn.Linear(128,10))
    
    def forward(self, input, h_t_1=None, c_t_1=None):
        input=input.view(batch_size,1,15,75)
        cnn_outputs = self.conv_layer(input)
        cnn_outputs=cnn_outputs.view(batch_size,1,200)
        rnn_outputs, (hn, cn) = self.recurrent_layer(cnn_outputs)
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits
model=SequenceClassify1()
optimizer_advanced=torch.optim.Adam(model.parameters(), lr=1e-3,amsgrad=True)
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 50
evaluate_every_epoch = 1
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer_advanced)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)  

C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 1.9872,  Accuracy: 27.00
val Epoch: 0  , Loss: 2.0342,  Accuracy: 24.50
train Epoch: 1  , Loss: 1.3395,  Accuracy: 55.30
val Epoch: 1  , Loss: 1.0904,  Accuracy: 61.25
train Epoch: 2  , Loss: 0.9735,  Accuracy: 66.75
val Epoch: 2  , Loss: 0.9435,  Accuracy: 65.75
train Epoch: 3  , Loss: 0.8066,  Accuracy: 71.45
val Epoch: 3  , Loss: 0.8793,  Accuracy: 67.75
train Epoch: 4  , Loss: 0.7472,  Accuracy: 72.75
val Epoch: 4  , Loss: 1.0320,  Accuracy: 64.50
train Epoch: 5  , Loss: 0.6718,  Accuracy: 75.95
val Epoch: 5  , Loss: 0.7636,  Accuracy: 73.75
train Epoch: 6  , Loss: 0.6545,  Accuracy: 75.50
val Epoch: 6  , Loss: 0.7589,  Accuracy: 75.25
train Epoch: 7  , Loss: 0.5936,  Accuracy: 78.55
val Epoch: 7  , Loss: 0.6453,  Accuracy: 76.00
train Epoch: 8  , Loss: 0.5524,  Accuracy: 79.70
val Epoch: 8  , Loss: 0.7369,  Accuracy: 74.50
train Epoch: 9  , Loss: 0.5510,  Accuracy: 79.80
val Epoch: 9  , Loss: 0.5934,  Accuracy: 80.50
train Epoch: 10  , Loss: 0.5087,  Accura

In [14]:
# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results_cnn5.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)
print(count)

1000


## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epochs) and any visualizations or graphs that you make in the process of training and evaluating your network.

Report your Kaggle Performance here:

I tried different types of architecture like Lstm,bidirectional LSTM, Lstm with convolutional network, tried various kernel sizes
in cnn layer and different number of lstm layers.
Final network architecture:
    I used convlutional layer, recurrent layer and classify layer.
    convolutional layer has the following architecture:
    - convolutional layer with 400 filters of size 2X2 and stride 1
    - Max pool layer of size 2X2 and stride 2
    - leakyrelu with alpha= 0.02
    - batch normalization
    - convolutional layer with 300 filters of size 2X2 and stride 1
    - Max pool layer of size 2X2 and stride 2
    - leakyrelu with alpha= 0.02
    - batch normalization
    - convolutional layer with 200 filters of size 2X2 and stride 1
    - Max pool layer of size 2X2 and stride 2
    - leakyrelu with alpha= 0.02
    - batch normalization
    - Flatten
    - linear layer with input 1600 and output size 200
            
    recurrent layer has the following parameters:
    - I used bidirectional lstm, with num of layers=2, dropout=0.2
    
    Classify layer has the following architecture:
    - Linear layer with input as 800 and output 400
    -leakyrelu layer with alpha 0.02
    -batchnormalization
    -linear layer with input as 400 and output as 128
    -leakyrelu with alpha 0.02
    -batchnormalization
    -linear layer with input as 128 and 10 outputs
        
        
        
optimizer: tried rmsprop and adam, finally used adam optimizer with learning rate=1e-3
epochs: tried many epochs, finally ran my model on 50 epochs and got training accuracy of around 96 % and validation accuracy of around  86 %.

Kaggle score :- 0.83
kaggle rank :- 8